In [8]:
#### pre-process ASB data
cell_line = "K562" # K562 or HepG2
rbp_dir = "/home/galaxy/project/alleleSpecific_analysis/data/jointAnalysis/AS/%s/ASB/" % cell_line
xy = "%s/%s_ASB_xy.tsv" % (rbp_dir, cell_line)
xxs = "%s/%s_xxs.txt" % (rbp_dir, cell_line)

def process_xy(in_file):
    out_file = in_file.split(".")[0] + "_hg19.bed"
    df = pd.read_table(in_file)
    df['start'] = df['position'] - 1
    df['or'] = ((df['ref_clip_count']+1)*(df['alt_rna_count']+1))/((df['alt_clip_count']+1)*(df['ref_rna_count']+1))
    df['mark'] = np.where(df['or']>1, "ref", "alt")
    df_res = df[['chrom','start','position','rbp','or','mark','ref_allele','alt_allele']]
    df_res.to_csv(out_file, sep="\t", index=False)

def process_xxs(in_file):
    out_file = in_file.split(".")[0] + "_hg19.bed"
    df = pd.read_table(in_file)
    df['start'] = df['Coordinate'] - 1
    df_res = df[['Chr','start','Coordinate','RBP','Major allele','Minor allele']]
    df_res.to_csv(out_file, sep="\t", index=False)

process_xy(xy)
process_xxs(xxs)

In [30]:
#### correlate with ASm6A

cell_line = "HepG2" # K562 or HepG2
rbp_dir = "/home/galaxy/project/alleleSpecific_analysis/data/jointAnalysis/AS/%s/ASB/" % cell_line
xy = "%s/%s_ASB_xy_hg19.bed" % (rbp_dir, cell_line)
xxs = "%s/%s_xxs_hg19.bed" % (rbp_dir, cell_line)

# asm6a = "/Charles/project/ASm6A/ASm6A/common_ASm6A/totaltissues/common_ASm6A.bed"
# result_dir = "/Charles/project/ASm6A/ASm6A/corrRBP/ASm6A_vs_ASB/common_ASm6A/%s/" % cell_line
# os.system("mkdir -p %s" % result_dir)
asm6a_dir = "/Charles/project/ASm6A/ASm6A/"
asm6a_list = glob.glob("%s/*.bed" % asm6a_dir)
result_dir = "/Charles/project/ASm6A/ASm6A/corrRBP/ASm6A_vs_ASB/each_sample/"

def stat_corr_results(in_file, out_file):
    df = pd.read_table(in_file)
    # rbp_list = list(set(df['RBP'].tolist()))
    rbp_list = ["FUS", "RBM15", "SLTM", "TRA2A", "HLTF", "AKAP1", "UPF1", "IGF2BP1", "CSTF2T", "YBX3", "ZNF622", "EIF3H", "ZC3H11A", "UCHL5"]
    res_list = []
    for rbp in rbp_list:
        df_i = df[df['RBP'] == rbp]
        df_i_con = df_i[df_i['rbp_mark'] == df_i['ASm6A_mark']]
        df_i_uncon = df_i[df_i['rbp_mark'] != df_i['ASm6A_mark']]
        res_list.append("\t".join([rbp, str(len(df_i_con)), str(len(df_i_uncon))]) + "\n")
    with open(out_file, 'w') as fw:
        fw.write("RBP\tconsistant\tinconsistant\n")
        fw.writelines(res_list)
            
        
def corr_xy(asm6a, rbp):
    result_file = os.path.join(result_dir, "%s-%s" % (os.path.basename(asm6a).split(".bed")[0], os.path.basename(rbp).replace(".bed",".txt")))
    os.system("bedtools intersect -a %s -b %s -wa -wb > %s" % (rbp, asm6a, result_file))
    if os.path.getsize(result_file):
        df = pd.read_table(result_file, header=None)
        df_sub = df.iloc[:,[3,5,11]]
        df_sub.columns = ["RBP","rbp_mark", "ASm6A_mark"]
        df_sub['ASm6A_mark'] = df_sub['ASm6A_mark'].str.split(";").str[2]
        count = result_file.replace(".txt",".count")
        df_sub.to_csv(count, sep="\t", index=False)
        stat_file = result_file.replace(".txt",".count.stat")
        stat_corr_results(count, stat_file)
    
    
def corr_xxs(asm6a, rbp):
    result_file = os.path.join(result_dir, "%s-%s" % (os.path.basename(asm6a).split(".bed")[0], os.path.basename(rbp).replace(".bed",".txt")))
    os.system("bedtools intersect -a %s -b %s -wa -wb > %s" % (rbp, asm6a, result_file))
    if os.path.getsize(result_file):
        df = pd.read_table(result_file, header=None)
        df_sub = df.iloc[:,[3,4,9]]
        df_sub.columns = ["RBP","major_allele","info"]
        ## assign mark to RBP
        df_sub['ASm6A_mark'] = df_sub['info'].str.split(";").str[2]
        df_sub['ref_allele'] = df_sub['info'].str.split(">").str[0]
        df_sub['rbp_mark'] = np.where(df_sub['major_allele']==df_sub['ref_allele'], "ref", "alt")
        df_sub = df_sub[['RBP','rbp_mark','ASm6A_mark']]
        
        count = result_file.replace(".txt",".count")
        df_sub.to_csv(count, sep="\t", index=False)
        stat_file = result_file.replace(".txt",".count.stat")
        stat_corr_results(count, stat_file)
    
for asm6a in asm6a_list:
    corr_xy(asm6a, xy)
    corr_xxs(asm6a, xxs)

/home/galaxy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/galaxy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/galaxy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

In [20]:
#### stat ASm6A number overlapping with 14 RBPs
rbp_dir = "/home/galaxy/data/RBP/ENCODE/20201124_2121/merged_bed/hg19/rename/union_peaks/"
# rbp_list = glob.glob("%s/*.bed" % rbp_dir)
name_list = ["FUS", "RBM15", "SLTM", "TRA2A", "HLTF", "AKAP1", "UPF1", "IGF2BP1", "CSTF2T", "YBX3", "ZNF622", "EIF3H", "ZC3H11A", "UCHL5"]
rbp_list = ["%s/%s.bed" % (rbp_dir, name) for name in name_list]

asm6a = "/Charles/project/ASm6A/ASm6A/common_ASm6A/totaltissues/common_ASm6A.bed"

for rbp in rbp_list:
    overlap_asm6a = os.popen("bedtools intersect -a %s -b %s -wa | sort -u | wc -l" % (asm6a, rbp)).read().split()[0]
    print(os.path.basename(rbp), overlap_asm6a)

FUS.bed 15
RBM15.bed 62
SLTM.bed 12
TRA2A.bed 14
HLTF.bed 10
AKAP1.bed 65
UPF1.bed 145
IGF2BP1.bed 31
CSTF2T.bed 9
YBX3.bed 25
ZNF622.bed 29
EIF3H.bed 14
ZC3H11A.bed 9
UCHL5.bed 54


In [23]:
#### stat ASm6A number overlapping with 14 RBPs
cell_line = "HepG2" # K562 or HepG2
rbp_dir = "/home/galaxy/project/alleleSpecific_analysis/data/jointAnalysis/AS/%s/ASB/" % cell_line
xy_1 = "%s/%s_ASB_xy_hg19.bed" % (rbp_dir, cell_line)
xxs_1 = "%s/%s_xxs_hg19.bed" % (rbp_dir, cell_line)

cell_line = "K562" # K562 or HepG2
rbp_dir = "/home/galaxy/project/alleleSpecific_analysis/data/jointAnalysis/AS/%s/ASB/" % cell_line
xy_2 = "%s/%s_ASB_xy_hg19.bed" % (rbp_dir, cell_line)
xxs_2 = "%s/%s_xxs_hg19.bed" % (rbp_dir, cell_line)

name_list = ["FUS", "RBM15", "SLTM", "TRA2A", "HLTF", "AKAP1", "UPF1", "IGF2BP1", "CSTF2T", "YBX3", "ZNF622", "EIF3H", "ZC3H11A", "UCHL5"]

asm6a = "/Charles/project/ASm6A/ASm6A/common_ASm6A/totaltissues/common_ASm6A.bed"

df_xy_1, df_xxs_1, df_xy_2, df_xxs_2 = pd.read_table(xy_1),pd.read_table(xxs_1), pd.read_table(xy_2), pd.read_table(xxs_2) 
print("\t".join(['RBP', 'XY_HepG2', 'XXS_HepG2', 'XY_K562', 'XXS_K562']))
for rbp in name_list:
    xy_1 = df_xy_1[df_xy_1['rbp'] == rbp]
    xxs_1 = df_xxs_1[df_xxs_1['RBP'] == rbp]
    xy_2 = df_xy_2[df_xy_2['rbp'] == rbp]
    xxs_2 = df_xxs_2[df_xxs_2['RBP'] == rbp]
    print("\t".join([rbp, str(len(xy_1)), str(len(xxs_1)), str(len(xy_2)), str(len(xxs_2))]))

RBP	XY_HepG2	XXS_HepG2	XY_K562	XXS_K562
FUS	0	32	0	164
RBM15	338	2	502	61
SLTM	183	1	550	71
TRA2A	69	4	313	4
HLTF	0	61	629	36
AKAP1	0	62	0	8
UPF1	0	107	391	69
IGF2BP1	251	39	284	5
CSTF2T	345	284	621	187
YBX3	0	49	779	107
ZNF622	0	0	658	103
EIF3H	272	0	0	0
ZC3H11A	0	0	0	0
UCHL5	667	46	992	136
